In [2]:
import pandas as pd
import datetime
import numpy as np
from sqlalchemy import create_engine
import pymysql
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from collections import OrderedDict
import boto3

In [96]:
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


# normalize the dataset
def prepare_data(series, n_lag, n_seq):
    # extract raw values
    raw_values = series.values

    # rescale values to -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(raw_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)

    # transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values

    return scaler, supervised_values


# create an LSTM network to training data
def create_lstm(X, y, n_lag, n_seq, n_batch, n_neurons):
    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


def fit_lstm(nb_epoch, model, n_batch):
    # fit network - reset state between epochs
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
        model.reset_states()


# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    model.reset_states()
    # convert to array
    return [x for x in forecast[0, :]]


# inverse data transform on forecasts
def inverse_transform(series, forecast, scaler):
    # create array from forecast
    forecast = np.array(forecast)
    forecast = forecast.reshape(1, -1)

    # invert scaling
    inv_scale = scaler.inverse_transform(forecast)
    inv_scale = inv_scale[0, :]

    return inv_scale


def extrapolate_timeslots(start_date, num_days=1):
    """Extrapolate the forecast dates and timeslots and return dataframe with forecast_dates and timeslots"""

    timeslots_per_day = 96
    forecast_dates = []
    timeslots = []

    for days in range(1, num_days + 1):
        forecast_dates = forecast_dates + [(start_date + datetime.timedelta(days=days)).date()] * timeslots_per_day
        timeslots = timeslots + list(range(1, timeslots_per_day + 1))

    dictionary = {
        "forecast_date": forecast_dates,
        "timeslot": timeslots
    }

    return pd.DataFrame.from_dict(dictionary)

In [4]:
# Definitions
db_host = "yieldmanagement.c2jaydssljuc.us-west-2.rds.amazonaws.com"
db_username = "ymdsmaster"
db_password = "YieLDMgMt36"
db_name = "yieldmanagement"
db_port = "6174"

conn = create_engine('mysql+pymysql://' + db_username + ':' + db_password + '@' + db_host + ':' + db_port + '/' + db_name, echo=False)

start_date = '2017-07-01'
end_date = '2017-12-31'
channel = 'ESPN'
network_number = "33"  # This is ESPN

In [5]:
# Extract data
query = (
    "SELECT network_label, start_time, household_impressions, timeslot FROM yieldmanagement.rentrak_impressions_old where start_date BETWEEN '" + start_date + "' AND '" + end_date +"' AND network_label IN ('" + channel + "') ORDER BY network_label, start_time;")

series = pd.read_sql(query, con=conn)

In [6]:
# Obtain column vector of values
series.set_index(keys=['start_time'], drop=False, inplace=True)
series.drop(['network_label', 'timeslot', 'start_time'], axis=1, inplace=True)
print("series shape", series.shape)
print("series columns", series.columns)
values = np.log(series.values)

('series shape', (17664, 1))
('series columns', Index([u'household_impressions'], dtype='object'))


In [9]:
series.head()

,household_impressions
start_time,
2017-07-01 00:00:00,45365
2017-07-01 00:15:00,43367
2017-07-01 00:30:00,42509
2017-07-01 00:45:00,39906
2017-07-01 01:00:00,42388


In [55]:
# Define lag and prediction windows
n_lag = 60 
n_seq = 10

In [56]:
# Easier for stateful LSTM
n_batch = 1

nb_epoch = 2
n_neuron = 128

In [57]:
# prepare data
scaler, train = prepare_data(series, n_lag, n_seq)
print(train.shape)

ValueError: Expected 2D array, got 1D array instead:
array=[ 45365.  43367.  42509. ...,  77018.  77797.  70678.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [41]:
######### TESTING #############
def prepare_data(series, n_lag, n_seq):
    # extract raw values
    raw_values = series.values

    # rescale values to -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(raw_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)

    # transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values

    return scaler, supervised_values

In [42]:
raw_values = series.values
print(type(raw_values))
print(raw_values.shape)

<type 'numpy.ndarray'>
(17664, 1)


In [43]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_values = scaler.fit_transform(raw_values)

In [44]:
scaled_values2.shape

(17664, 1)

In [45]:
supervised = series_to_supervised(scaled_values, n_lag, n_seq)

In [46]:
cols, names = list(), list()

In [47]:
n_in = n_lag
n_out = n_seq
n_vars = 1
dropnan = True
for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
# put it all together
agg = pd.concat(cols, axis=1)
agg.columns = names
# drop rows with NaN values
if dropnan:
    agg.dropna(inplace=True)

In [82]:
cols, names = list(), list()
n_in = n_lag #60
n_out = n_seq #10
n_vars = 1
dropnan = True

In [83]:
########### TESTING ############
## Accepts a numpy array as data input
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [84]:
supervised = series_to_supervised(scaled_values,n_in=n_in,n_out=n_out,dropnan=dropnan)

In [94]:
######### Get caught up ###########
db_host = "yieldmanagement.c2jaydssljuc.us-west-2.rds.amazonaws.com"
db_username = "ymdsmaster"
db_password = "YieLDMgMt36"
db_name = "yieldmanagement"
db_port = "6174"

conn = create_engine('mysql+pymysql://' + db_username + ':' + db_password + '@' + db_host + ':' + db_port + '/' + db_name, echo=False)

start_date = '2017-07-01'
end_date = '2017-12-31'
channel = 'ESPN'
network_number = "33"  # This is ESPN

query = (
    "SELECT network_label, start_time, household_impressions, timeslot FROM yieldmanagement.rentrak_impressions_old where start_date BETWEEN '" + start_date + "' AND '" + end_date +"' AND network_label IN ('" + channel + "') ORDER BY network_label, start_time;")

series = pd.read_sql(query, con=conn)

series.set_index(keys=['start_time'], drop=False, inplace=True)
series.drop(['network_label', 'timeslot', 'start_time'], axis=1, inplace=True)
print("series shape", series.shape)
print("series columns", series.columns)
values = np.log(series.values)

n_lag = 30 * 96
n_seq = 7 * 96

# Easier for stateful LSTM
n_batch = 1

nb_epoch = 2
n_neuron = 128

('series shape', (17664, 1))
('series columns', Index([u'household_impressions'], dtype='object'))


In [103]:
scaler, train = prepare_data(series, n_lag, n_seq)

In [109]:
# reshape training into [samples, timesteps, features]
X, y = train[:, 0:n_lag], train[:, n_lag:]
X = X.reshape(X.shape[0], 1, X.shape[1])

In [112]:
y.shape

(14113, 672)

In [113]:
# Create model
model = create_lstm(X, y, n_lag, n_seq, n_batch, n_neuron)
# create an LSTM network to training data
#def create_lstm(X, y, n_lag, n_seq, n_batch, n_neurons):
#    # design network
#    model = Sequential()
#    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
#    model.add(Dense(y.shape[1]))
#    model.compile(loss='mean_squared_error', optimizer='adam')
#    return model

In [114]:
# Fit LSTM
def fit_lstm(nb_epoch, model, n_batch):
    # fit network - reset state between epochs
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
        model.reset_states()

In [115]:
nb_epoch = 1
fit_lstm(nb_epoch, model, n_batch)

Epoch 1/1
14113/14113 [==============================] - 255s - loss: 0.0399   


In [133]:
# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    model.reset_states()
    # convert to array
    return [x for x in forecast[0, :]]

In [134]:
forecast = forecast_lstm(model, X, n_batch)

In [152]:
rescaled_forecast = inverse_transform(series, forecast, scaler)

In [136]:
f = forecast

In [138]:
f = np.array(f)

In [143]:
f = f.reshape(1,-1)

In [145]:
inv_scale = scaler.inverse_transform(f)

In [148]:
inv_scale = inv_scale[0, :]

In [159]:
t = scaler.inverse_transform(train[14112,:].reshape(1,-1))

In [165]:
rows = len(rescaled_forecast)

In [166]:
# inverse data transform on forecasts
def inverse_transform(series, forecast, scaler):
    # create array from forecast
    forecast = np.array(forecast)
    forecast = forecast.reshape(1, -1)

    # invert scaling
    inv_scale = scaler.inverse_transform(forecast)
    inv_scale = inv_scale[0, :]

    return inv_scale

In [167]:
create_date = datetime.date.today()

In [172]:
# Last date of our impression data we need to forecast from
last_date = datetime.datetime(2017, 12, 31, 0, 0)
timeslot_df = extrapolate_timeslots(last_date, 7)

In [175]:
forecast_dict = OrderedDict([
    ('create_date', [create_date] * rows),
    ('forecast_date', timeslot_df['forecast_date']),
    ('data_source_marker', ['R'] * rows),      # Rentrak
    ('network_id', [network_number] * rows),
    ('master_network_id', [''] * rows),
    ('timeslot', timeslot_df['timeslot']),
    ('impression_type', ['H'] * rows),        # Households
    ('forecast_impressions', rescaled_forecast.astype(int))
])

In [170]:
forecast_dict

OrderedDict([('create_date',
              [datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.date(2018, 3, 14),
               datetime.d

In [171]:
def extrapolate_timeslots(start_date, num_days=1):
    """Extrapolate the forecast dates and timeslots and return dataframe with forecast_dates and timeslots"""

    timeslots_per_day = 96
    forecast_dates = []
    timeslots = []

    for days in range(1, num_days + 1):
        forecast_dates = forecast_dates + [(start_date + datetime.timedelta(days=days)).date()] * timeslots_per_day
        timeslots = timeslots + list(range(1, timeslots_per_day + 1))

    dictionary = {
        "forecast_date": forecast_dates,
        "timeslot": timeslots
    }

    return pd.DataFrame.from_dict(dictionary)

In [173]:
type(timeslot_df)

pandas.core.frame.DataFrame

In [177]:
rescaled_forecast.astype(int)

array([ 301289,  300934,  296004,  287443,  275149,  259812,  241785,
        221735,  199841,  176315,  152017,  126913,  102727,   79011,
         56489,   35598,   17459,     768,  -13230,  -25517,  -34391,
        -40441,  -44722,  -46260,  -44839,  -41993,  -36828,  -30366,
        -22235,  -12637,   -3090,    8327,   19764,   31088,   43466,
         55215,   66836,   78334,   88640,   98998,  108409,  116154,
        122618,  126933,  129989,  131492,  130931,  129954,  129299,
        130550,  132790,  135945,  141435,  148243,  156736,  166503,
        176912,  190097,  202717,  213861,  221978,  226782,  227237,
        223641,  215806,  204879,  190423,  173910,  154929,  134532,
        113246,   91696,   71291,   51434,   33562,   16760,    2809,
         -8831,  -13973,  -10670,    3780,   31551,   74868,  132150,
        200208,  276989,  360075,  447264,  532567,  615214,  695108,
        770505,  839761,  902761,  958206,  998844, 1019715, 1022228,
       1006956,  975